# julia itteration solver sparse src

In [3]:
mutable struct flop_counter
    f::Float64
end

In [2]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate
using SparseArrays, Arpack, LinearAlgebra
using BenchmarkTools, IterativeSolvers, MatrixDepot, Random


struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]


struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct FastUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct OffDiagonal{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

function backward_sub!(F::FastUpperTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = A.n : -1 : 1

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = x[col] / A.nzval[idx]

        # Substitute next values involving x[col]
        for i = A.colptr[col] : idx - 1
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end


function forward_sub!(fp::flop_counter, F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        fp.f += 1
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
            fp.f += 2
        end
    end
    x
end

function backward_sub!(fp::flop_counter, F::FastUpperTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = A.n : -1 : 1

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = x[col] / A.nzval[idx]
        fp.f += 1
        # Substitute next values involving x[col]
        for i = A.colptr[col] : idx - 1
            x[A.rowval[i]] -= A.nzval[i] * x[col]
            fp.f += 2
        end
    end

    x
end




include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


backward_sub! (generic function with 2 methods)

## Matrix Generator

In [5]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random


using LinearAlgebra
using SparseArrays

function cartesianidx(k::Int, n::Int)
    """
    {s_i} are on a regular 10 by 10 lattice over the two dimensional domain
    S = [1, 10] \times [1, 10].
    output gives a cartesian index (i, j) of s_k, i.e., 
    S = [s01 s11 s21 ... s91
         s02 s12 s22 ... s92
         s03 s13 s23 ... s93
            ... ... 
         s10 s20 s30 ... s100]
    """
    if k % n == 0
        [n; k ÷ n]
    else
        [k % n; k ÷ n + 1]
    end
end

"""
n_i is the number of points neighbouring s_i, i.e., with distance 1from s_i
N[i, j] = n_k, where cartesianindex(k, 10) = (i, j)
"""

function gen_Ab(n)
    #n = 10
    N = 4 * ones(n, n);
    for k in 1:n
        N[1, k] -= 1
        N[k, 1] -= 1
        N[n, k] -= 1
        N[k, n] -= 1
    end

    """
    6.1 A 10 by 10 lattice example (n = 100)
    """
    A = zeros(n^2, n^2)
    for i in 1:n^2
        for j in 1:n^2
            if i == j
                A[i, j] = 0.0001 + N[i]
            elseif norm(cartesianidx(i, n)-cartesianidx(j, n)) <= 1.0
                A[i, j] = -1.0
            else
                A[i, j] = 0.0
            end
        end
    end

    A = sparse(A)
    b = randn(n^2)
    
    (A,b)
end



function gen_Ab_temp(n)
    #n = 100
    # Poisson matrix of dimension n^2=10000, pd and sparse
    A = matrixdepot("poisson", n)
    @show typeof(A)
    # dense matrix representation of A
    #Afull = convert(Matrix, A)
    #@show typeof(Afull)
    # sparsity level
    count(!iszero, A) / length(A)
    b = randn(n^2)
    (A,b)
end

gen_Ab_temp (generic function with 1 method)

# K3 src

In [1]:
mutable struct flop_counter
    f::Float64
end

# SOR

In [10]:
function f_mul!(fp::flop_counter, α::T, O::DiagonalIndices, x::AbstractVector, b::AbstractVector, r::AbstractVector  ) where {T}
    A = O.matrix
    r[:] = b
    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : A.colptr[col + 1] - 1
            r[A.rowval[j]] += A.nzval[j] * αx 
            fp.f +=2
        end
        fp.f +=1
    end
    r
end

function sum!(fp::flop_counter, z, r, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        z[i] += r[i]
        fp.f +=1
    end
end


function m_sor!(fp::flop_counter,A, D::DiagonalIndices, w)
    for d_idx in D.diag 
        A.nzval[d_idx]  *= (1/w)
        fp.f +=2
    end
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_sor!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    fp = flop_counter(0)
    
    for i = 1 : max_itter 
        f_mul!(fp, -one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("sor_itter : ",i)
            break
        end 
        
        forward_sub!(fp,F, r)# r <- M_sor\r
        sum!(fp,x, r, A) # x <- x +  M_sor/b 
        print(fp.f)
    end
    x
    
end

function k3_sor(A, b::AbstractVector, w, maxiter)
    fp_1 = flop_counter(0)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    m_sor!(fp_1, m_sor, D, w)
    D_ms = DiagonalIndices(m_sor)
    itter_sor!(FastLowerTriangular(m_sor ,D_ms), D, x , b, maxiter)
    print(fp_1.f)
end



k3_sor (generic function with 1 method)

In [8]:
A, b = gen_Ab(10)
b_ = copy(b)

100-element Array{Float64,1}:
 -1.7315758115819326  
 -0.07997027726456443 
 -0.8717545788440997  
  0.7733603550405481  
 -1.23025949718485    
 -0.6795623171595335  
 -0.010805773536428006
  1.0184230151812865  
  1.2922470361893663  
 -0.83611955576624    
  0.5521943322533005  
  0.05351062498107297 
 -1.0419308334324373  
  ⋮                   
  1.432654869366426   
 -0.8308116881216856  
 -0.49050481966992765 
 -0.8752173846676085  
  0.41643854460137203 
 -0.16597548367272988 
 -1.0381618314231047  
  0.3442747472946145  
  0.9066603636859343  
  1.851695590343076   
  0.8408877379738464  
  0.013748637305547661

In [11]:
sol_6 = k3_sor(A, b_, 1.9852, 1)

1580.0200.0

# SSOR

In [17]:
function gamma_sqrt_diag_mul!(fp::flop_counter, D::DiagonalIndices, b::AbstractVector, w ,b_c)
    A = D.matrix
    for idx in D.diag 
        b[A.rowval[idx]] *=  sqrt( b_c * ((2/w) -1) * A.nzval[idx])
        fp.f += 6
    end
end

function sum2!(fp::flop_counter, x,y,z, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        x[i] =y[i]+z[i]
        fp.f += 1
    end
end

function itter_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    #symetric일때도 필요한지 고려 diag정의는 새로필요한거 같음

    fp = flop_counter(0)

    T = eltype(b)
    r = zeros(A.n)
    y = zeros(A.n)
        
    for i = 1 : max_itter 
        f_mul!(fp, -one(T), D, x, b, r) # r_1 <-  γ * D^(1/2) * b- Ay
        
        if norm(r) < 10^(-8)
            println("ssor_itter : ",i)
            break
        end 
        
        gamma_sqrt_diag_mul!(fp, D,r,w,1)
        forward_sub!(fp, F, r) #r_1 <- m_sor\r_1
        gamma_sqrt_diag_mul!(fp, D,r,w,1)
        backward_sub!(fp, U, r)
        sum!(fp, x, r, A)
        println("for 1 itter :", fp.f)
    end
    x
end



function k3_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    fp_1 = flop_counter(0)

    
    m_sor!(fp_1, m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    
    println("for inital :", fp_1.f)
    itter_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, maxiter)
end


k3_ssor (generic function with 1 method)

In [18]:
b_ = copy(b)
sol_5 = k3_ssor(A, b_, 1.9852, 1)

for inital :200.0
for 1 itter :3240.0

100-element Array{Float64,1}:
 -1.021620652489922   
 -0.5493349596819     
 -0.5253570242229018  
 -0.4702521834581398  
 -0.4160397339451868  
 -0.33671469506439156 
 -0.24145715457646325 
 -0.15178279379561777 
 -0.08773851573319826 
 -0.05874088950512483 
 -0.45432618649381923 
 -0.2828636902985248  
 -0.2962565871469309  
  ⋮                   
 -0.0623261060160197  
 -0.07793466133992544 
 -0.041650564834754475
 -0.02957829698164326 
 -0.026823263339203776
 -0.041889363974324065
 -0.06440633170311852 
 -0.06658695859164611 
 -0.05202461496238094 
 -0.03758608421473334 
 -0.03889644250255729 
 -0.05436254119675141 

# Cheby_SSOR

In [33]:
mutable struct CB_variable
    β::Float64
    α::Float64
    b::Float64
    a::Float64
    κ::Float64
end

function mul_sum!(fp::flop_counter,x_next, x, x_pre, r, α, τ, A)
    for i = 1:A.n 
        x_next[i] = (1-α)*x_pre[i] + α*x[i] + (τ*α*r[i])
        fp.f += 6
    end
    # x
end



function sum2!(fp::flop_counter,x_next, x, r, A)
    for i = 1:A.n 
        x_next[i] = x[i] + r[i]
        fp.f += 1
    end
    # x
end

function sum3!(fp::flop_counter, w_v, x_temp , x, r, A)
    for i = 1:A.n 
        w_v[i] = x_temp[i] - x[i] + r[i]
        fp.f += 2
    end
    # x
end

function eigMm(A::SparseMatrixCSC, ω::Real)

    Dw = sqrt((2/ω-1)) * Diagonal(sqrt.(diag(A)))
    L = (LowerTriangular(A)- (1-1/ω) * Diagonal(A))*(inv(Dw))

    Meig = inv(cholesky(L*L')) * A
    
    λ_max = eigs(Meig; nev=1, ritzvec=false, which=:LM)[1][1]
    λ_min = eigs(Meig; nev=1, ritzvec=false, which=:SM)[1][1]
    real(λ_max), real(λ_min)
end


function itter_CB_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  λ_max, λ_min, max_itter)
 
    fp = flop_counter(0)

    A = D.matrix
    A_t = D_t.matrix
    
    δ = ((λ_max - λ_min)/4)^2
    τ = 2/(λ_max + λ_min)
    
    T = eltype(b)
    cb = CB_variable(0,0,0,0,0)
    #Assign initial parameter
    cb.β  = 2*τ
    cb.α = 1
    cb.b = 2/cb.α - 1
    cb.a = (2/τ -1) * cb.b
    cb.κ = τ
    
    T = eltype(b)
    r_1 = zeros(A.n)
    r_2 = zeros(A.n)
    x_pre = zeros(A.n)
    x_next = zeros(A.n)
    x_temp = zeros(A.n)
    w_v = zeros(A.n)
 
    for i = 1 : max_itter 

        x_pre[:] = x 
        x[:] = x_next
        
        f_mul!(fp, -one(T), D, x, b, r_1) # r <- b - A* X        
        """
        if norm(r_1) < 10^(-8)
            #println("CB sor_itter : ",i)
            break
        end 
        
        """
        
        
        
        forward_sub!(fp, F, r_1) #r_1 <- m_sor\r_1
        sum2!(fp, x_temp, x, r_1, A) #x_next <- x + τ*r
        
        f_mul!(fp, -one(T), D, x_temp, b, r_2) # r_2 <- b - A* X
        backward_sub!(fp, U, r_2)
        sum3!(fp, w_v, x_temp , x, r_2, A)
        
        if i == 1
            sum2!(fp, x_next, cb.α*x, τ*w_v, A) #x_next <- x + τ*r
        else
            mul_sum!(fp, x_next, x, x_pre, w_v, cb.α, τ, A) # x_next <- (1-α)*x_pre + α*x + (τ*α*r[i])
        end
        
        
        cb.β = 1 / ( (1/τ)  - cb.β*δ ) 
        cb.α = cb.β / τ
        cb.b = ( ( 2*cb.κ*(1- cb.α) ) / cb.β) + 1
        cb.a = ((2/τ) -1) + (cb.b-1) * ( (1/τ) + (1/cb.κ) -1 )
        cb.κ = cb.β  + ( (1 - cb.α) * cb.κ)
        
        println("for 1 itter :", fp.f)
        
    end
    x

end



function k3_CB_ssor(A, b::AbstractVector, w, λ_max, λ_min,  maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    fp_1 = flop_counter(0)
    
    m_sor!(fp_1, m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    #λ_max,λ_min = eigMm(A, w)
    println("for inital :", fp_1.f)

    itter_CB_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, λ_max, λ_min, maxiter)
end


k3_CB_ssor (generic function with 1 method)

In [34]:
w =  1.9852
λ_max,λ_min = eigMm(A, w)
b_ = copy(b)
sol_5 = k3_CB_ssor(A, b_, 1.9852, λ_max,λ_min, 1)

for inital :200.0
for 1 itter :3360.0


100-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# Richardson

In [35]:

function itter_Richardson!(D::DiagonalIndices, x::AbstractVector, w, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    fp = flop_counter(0)
    for i = 1 : max_itter 
        f_mul!(fp, -one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Richardson_itter : ",i)
            break
        end 
        
        sum!(fp, x, w*r, A) # x <- x +  wr

        println("for 1 itter :", fp.f)
    end
    x
    
end


function k3_RCS(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_Richardson!(D, x , w, b, maxiter)
end



k3_RCS (generic function with 1 method)

In [38]:

b_ = copy(b)
sol_3 = k3_RCS(A, b_, 1, 1)


for 1 itter :1120.0


100-element Array{Float64,1}:
 -1.7315758115819326  
 -0.07997027726456443 
 -0.8717545788440997  
  0.7733603550405481  
 -1.23025949718485    
 -0.6795623171595335  
 -0.010805773536428006
  1.0184230151812865  
  1.2922470361893663  
 -0.83611955576624    
  0.5521943322533005  
  0.05351062498107297 
 -1.0419308334324373  
  ⋮                   
  1.432654869366426   
 -0.8308116881216856  
 -0.49050481966992765 
 -0.8752173846676085  
  0.41643854460137203 
 -0.16597548367272988 
 -1.0381618314231047  
  0.3442747472946145  
  0.9066603636859343  
  1.851695590343076   
  0.8408877379738464  
  0.013748637305547661

# Jacobi

In [39]:
function mul_inv_d!(fp::flop_counter, D, r)
    A = D.matrix
    for idx in D.diag 
        r[A.rowval[idx]] *=  (1 / A.nzval[idx])
        fp.f += 2
    end
end
    

function itter_JCB!(D::DiagonalIndices, x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    fp = flop_counter(0)
    for i = 1 : max_itter 
        f_mul!(fp, -one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Jacobi_itter : ",i)
            break
        end 

        mul_inv_d!(fp, D, r)# r <- D^-1*r
        sum!(fp, x, r, A) # x <- x +  M^-1*r
        
        println("for 1 itter :", fp.f)

    end
    x
    
end


function k3_JCB(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_JCB!(D, x , b, maxiter)
end



k3_JCB (generic function with 1 method)

In [40]:
b_ = copy(b)
sol_4 = k3_JCB(A, b_, 1)

for 1 itter :1320.0


100-element Array{Float64,1}:
 -0.8657446185600381   
 -0.0266558705591695   
 -0.290575173775574    
  0.2577781924071024   
 -0.41007282996728434  
 -0.22651322194577964  
 -0.0036018044519942685
  0.33946302295966346  
  0.430734654241314    
 -0.418038875939323    
  0.18405864212969583  
  0.01337732181222294  
 -0.26047619645319803  
  ⋮                    
  0.35815476347251973  
 -0.27692799844061383  
 -0.24524014782757242  
 -0.2917294039090725   
  0.13880822125974868  
 -0.05532331711367283  
 -0.34604240906073286  
  0.11475442395073979  
  0.3022100475603927   
  0.617211289738034    
  0.2802865697722897   
  0.006873974954026129 

# Gauss-Seidal

In [47]:

function m_GS!(fp::flop_counter, A, D::DiagonalIndices)
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_GS!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    fp = flop_counter(0)
    
    for i = 1 : max_itter 
        f_mul!(fp, -one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Gauss_Seidal_itter : ",i)
            break
        end 
        
        forward_sub!(fp, F, r)# r <- M_sor\r
        sum!(fp, x, r, A) # x <- x +  M_sor/b        
        
        println("for 1 itter :", fp.f)

    end
    x
    
end


function k3_GS(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    m_GS = copy(A)
    D = DiagonalIndices(A)
    fp_1 = flop_counter(0)
    m_GS!(fp_1, m_GS, D)
    println("for inital :", fp_1.f)

    D_GS = DiagonalIndices(m_GS)
    itter_sor!(FastLowerTriangular(m_GS ,D_GS), D, x , b, maxiter)

end



k3_GS (generic function with 1 method)

In [48]:
b_ = copy(b)
sol_4 = k3_GS(A, b_, 1)

for inital :0.0
1580.0

100-element Array{Float64,1}:
 -0.8657446185600383  
 -0.315227791015167   
 -0.3956476016996989  
  0.12590005444513488 
 -0.36810754399510515 
 -0.3492116466633241  
 -0.12000180667302827 
  0.2994637540442846  
  0.5305525783252728  
 -0.15277584992798718 
 -0.1045132783263017  
 -0.09155532220704377 
 -0.38227388248773286 
  ⋮                   
  0.3152352895220627  
 -0.24494436890715673 
 -0.4227354907081954  
 -0.4157062431295673  
  0.0926272029125421  
  0.037779787711954504
 -0.40751183646561057 
 -0.17870870045265638 
  0.21546113639977107 
  0.6647657697800938  
  0.6069427010019676  
  0.18786409149560443 

# Test

In [86]:
@benchmark ssor(A, b_, 1.6641, maxiter= 64000)

BenchmarkTools.Trial: 
  memory estimate:  3.91 MiB
  allocs estimate:  256010
  --------------
  minimum time:     174.948 ms (0.00% GC)
  median time:      182.340 ms (0.00% GC)
  mean time:        185.055 ms (1.34% GC)
  maximum time:     226.584 ms (22.32% GC)
  --------------
  samples:          28
  evals/sample:     1

In [89]:
w = 1.6641
λ_max,λ_min = eigMm(A, w)
@benchmark k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 1100)

BenchmarkTools.Trial: 
  memory estimate:  36.89 KiB
  allocs estimate:  33
  --------------
  minimum time:     3.112 ms (0.00% GC)
  median time:      3.273 ms (0.00% GC)
  mean time:        3.337 ms (1.31% GC)
  maximum time:     54.598 ms (93.92% GC)
  --------------
  samples:          1497
  evals/sample:     1

In [90]:
w = 1.6641
λ_max,λ_min = eigMm(A, w)
@benchmark k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 80000)

BenchmarkTools.Trial: 
  memory estimate:  36.89 KiB
  allocs estimate:  33
  --------------
  minimum time:     236.739 ms (0.00% GC)
  median time:      240.880 ms (0.00% GC)
  mean time:        243.286 ms (0.00% GC)
  maximum time:     263.917 ms (0.00% GC)
  --------------
  samples:          21
  evals/sample:     1

In [85]:
sol_1 = sor(A,b , 1.9852 ,maxiter=100000)
sol_2 = ssor(A,b , 1.6641 ,maxiter=100000)


#b_ = copy(b)
#sol_3 = k3_RCS(A, b_, 1, 100000)


b_ = copy(b)
sol_4 = k3_JCB(A, b_, 1000000)


b_ = copy(b)
sol_5 = k3_ssor(A, b_, 1.6641, 100000)

b_ = copy(b)
sol_6 = k3_sor(A, b_, 1.9852, 100000)

b_ = copy(b)
sol_7 = k3_CB_ssor(A, b_, 1, 100000)

b_ = copy(b)
sol_8 = k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 80000)

Jacobi_itter : 680053
ssor_itter : 63903
sor_itter : 1725
CB sor_itter : 1017
CB sor_itter : 638


100-element Array{Float64,1}:
 -1115.024659132122 
 -1115.0836624743986
 -1114.573890655761 
 -1114.7073389924383
 -1114.1214348944284
 -1113.559149202011 
 -1113.6090264340335
 -1113.7986450854858
 -1113.3337172177296
 -1113.3125049903535
 -1115.8260199168164
 -1115.7777208225439
 -1115.101501247324 
     ⋮              
 -1117.580131117082 
 -1118.060776607826 
 -1120.5001640232185
 -1120.5906476996554
 -1120.0388094047973
 -1119.9393941354429
 -1119.6723138345092
 -1119.052709286683 
 -1118.8129249830072
 -1118.036148446878 
 -1116.9604920304675
 -1118.0260757030715

In [83]:
sol_6

100-element Array{Float64,1}:
 -1115.0246603755354
 -1115.0836637200284
 -1114.5738919030093
 -1114.7073402403676
 -1114.121436144546 
 -1113.5591504541803
 -1113.6090276880068
 -1113.7986463414118
 -1113.333718475241 
 -1113.3125062502306
 -1115.8260211619456
 -1115.7777220693695
 -1115.101502496146 
     ⋮              
 -1117.5801323843825
 -1118.060777876408 
 -1120.500165278496 
 -1120.5906489563533
 -1120.0388106632456
 -1119.9393953954598
 -1119.6723150965565
 -1119.0527105505548
 -1118.812926248478 
 -1118.036149713508 
 -1116.9604932990276
 -1118.0260769732374

In [23]:
sol_4

10000-element Array{Float64,1}:
  0.34491546859697686  
 -0.23558888716138957  
  0.11987784530725726  
  0.061503776626506614 
  0.4064547871727671   
 -0.583143818730661    
 -0.14439710806145784  
 -0.14035840652493967  
 -0.1556705390981974   
  0.13210616732317065  
  0.5432297571465943   
 -0.0051115843476727905
  0.4467627060253428   
  ⋮                    
  0.5524034380129357   
  0.09440911968003189  
  0.05395113042203955  
  0.34440455505834605  
 -0.15987906883379494  
 -0.07044970309366055  
 -1.092756019743297    
 -0.7803666451224219   
 -0.8586800310875652   
 -0.37267263994381894  
 -0.4117121774759638   
 -0.3944917172390592   